In [1]:
!ls data/han/AWS/

남이섬_2016.xlsx  남이섬_2020.xlsx  청평_2019.xlsx  화천_2018.xlsx
남이섬_2017.xlsx  청평_2016.xlsx    청평_2020.xlsx  화천_2019.xlsx
남이섬_2018.xlsx  청평_2017.xlsx    화천_2016.xlsx  화천_2020.xlsx
남이섬_2019.xlsx  청평_2018.xlsx    화천_2017.xlsx


In [2]:
!pwd

/home/cjinw/work/water-quality/rnn


In [3]:
%matplotlib widget
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt
    
from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [4]:
# for Korean in plot
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [6]:

# folder = 'data/han/자동'
# folder = 'data/han/수질'
# folder = 'data/han/AWS'
# folder = 'data/nakdong/자동'
folder = 'data/gum/자동'
# folder = 'data/yeongsan1/자동'

# ## 한강
# ## 자동
# file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
#               ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
#               ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

## 수질
# file_names = [['가평천3_2016.xlsx','가평천3_2017.xlsx','가평천3_2018.xlsx', '가평천3_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천댐1_2016.xlsx','춘천댐1_2017.xlsx','춘천댐1_2018.xlsx', '춘천댐1_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['의암_2016.xlsx','의암_2017.xlsx','의암_2018.xlsx', '의암_2019.xlsx'],
#               ['춘성교_2016.xlsx','춘성교_2017.xlsx','춘성교_2018.xlsx', '춘성교_2019.xlsx'],
#               ['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천_2016.xlsx','춘천_2017.xlsx','춘천_2018.xlsx', '춘천_2019.xlsx']
#              ]


## AWS
# file_names = [['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2018.xlsx'],
#               ['청평_2016.xlsx', '청평_2017.xlsx', '청평_2018.xlsx', '청평_2019.xlsx'],
#               ['화천_2016.xlsx', '화천_2017.xlsx', '화천_2018.xlsx', '화천_2019.xlsx']]
 

    
## 낙동강
#자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]


## 금강
#자동
file_names = [['현도_2016.xlsx','현도_2017.xlsx','현도_2018.xlsx', '현도_2019.xlsx'],
              ['대청호_2016.xlsx','대청호_2017.xlsx','대청호_2018.xlsx', '대청호_2019.xlsx'],
              ['옥천천_2016.xlsx','옥천천_2017.xlsx','옥천천_2018.xlsx', '옥천천_2019.xlsx'],
              ['장계_2016.xlsx','장계_2017.xlsx','장계_2018.xlsx', '장계_2019.xlsx']]
             

    
    
## 영산강
#자동
# file_names = [['나주_2016.xlsx','나주_2017.xlsx','나주_2018.xlsx', '나주_2019.xlsx']]
# #               ['지석천_2016.xlsx','지석천_2017.xlsx','지석천_2018.xlsx', '지석천_2019.xlsx'],
# #               ['용봉_2016.xlsx','용봉_2017.xlsx','용봉_2018.xlsx', '용봉_2019.xlsx'],
#               ['서창교_2016.xlsx','서창교_2017.xlsx','서창교_2018.xlsx', '서창교_2019.xlsx'],
#               ['우치_2016.xlsx','우치_2017.xlsx','우치_2018.xlsx', '우치_2019.xlsx']]


In [7]:
target_col = '용존산소'
# 각각의 강 마다, 측정소 별로 column을 맞춰야함
# 0: 자동, 1: 수질, 2: AWS(ASOS), 
han = [':,2:', ':,28:45', ':,26:29']
nakdong = [':,2:', ':,2:12']
gum = [':,2:'] #, ':,28:45', ':,26:29']
# gain_han = ['save/han/1', 'save/han/2', 'save/han/3']

# interpolation_option = True
interpolation_option = False
# gain_save_path = gain_han[2]

iloc_val = gum[0]
iloc_val

':,2:'

In [8]:
def make_columns(df):
    column_list = ['측정날짜', '측정소명', '수온', '수소이온농도','전기전도도', '용존산소', '총유기탄소', '총질소', '총인', '클로로필-a']
    list_df = pd.DataFrame(columns=column_list)
    list_df
    df = df.drop(columns=df.columns.difference(column_list))
    new_column = list_df.columns.difference(df.columns)
#     print(new_column)
    if not new_column.empty :
        print("Make_columns")
        df[new_column] = pd.Series()
#     print('columns')
#     print(df.columns)
    return df

In [9]:
def make_timeseries(df, interpolate=None):
    print(df.shape)
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df[df[date_col].notna()]

    year = pd.DatetimeIndex(df[date_col]).year.astype(np.int64)

    start = str(year[0]) + "-01-01 00:00"
#     start

    end = str(year[0]) + "-12-31 23:00"
#     end

    time_series = pd.date_range(start = start, end = end, freq='H')
    time_series
    time_series = pd.DataFrame(time_series)
    time_series.columns = [date_col]   
    time_series

    df = pd.concat([time_series, df], axis=0)
    df = df.drop_duplicates([date_col], keep="last")
    df = df.sort_values([date_col], axis=0)
    
    if interpolate :
        print('interpolation')
        df = df.interpolate(metod='spline', order=2, limit_direction='both')
        
    print(df.shape)
    return df

In [10]:
def make_dataframe(file_names, iloc_val, interpolate=None):
    day = 24*60*60
    year = (365.2425)*day

    df_full = []
    df = []
    
    for loc in range(len(file_names)):
    
        df_loc = []
        for y in range(len(file_names[loc])):
            path = os.path.join(folder, file_names[loc][y])
            print(file_names[loc][y])
            df_tmp = make_timeseries(pd.read_excel(path), interpolate=interpolate)
#             df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
            df_tmp = make_columns(df_tmp)
            
    #         df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
    #         print(df_tmp.head) 
            df_loc.append(df_tmp)
#             df_loc.append(pd.read_excel(path))
        df_full.append(pd.concat(df_loc))
    #     df.append(df_full[loc].iloc[:,2:11])
        df.append(eval(f"df_full[loc].iloc[{iloc_val}]"))
    #     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
    #     df.append(df_full[loc].iloc[:, 2:11])
        date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
        timestamp_s = date_time.map(datetime.datetime.timestamp)
        df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
        df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
        df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
        df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
        df[loc] = df[loc].reset_index(drop=True)
        
#         print(df[loc].shape[0])
#         rows = df[loc].shape[0]
#         percentage = round(rows * 0.7)
#         df[loc] = df[loc].dropna(axis='columns',  thresh=percentage)
        

    
    return df

In [11]:
def normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_mean = df_all.mean()
    train_std = df_all.std()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_mean)/train_std
    
    
    return df_all, train_mean, train_std

In [12]:
def min_max_normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_min = df_all.min()
    train_max = df_all.max()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_min)/(train_max - train_min)       
    
    
    return df_all, train_min, train_max

In [13]:
##for normal
# df = make_dataframe(file_names, iloc_val)
## for interpolation
df = make_dataframe(file_names, iloc_val, interpolate=interpolation_option)
# df_all, train_mean, train_std = normalize(df)
df_all, train_mean, train_std = min_max_normalize(df)

현도_2016.xlsx
(8784, 24)
(8784, 24)
현도_2017.xlsx
(8760, 24)
(8760, 24)
현도_2018.xlsx
(8756, 24)
(8760, 24)
현도_2019.xlsx
(8760, 24)
(8760, 24)
대청호_2016.xlsx
(8088, 19)
(8784, 19)
대청호_2017.xlsx
(8736, 19)
(8760, 19)
대청호_2018.xlsx
(8760, 19)
(8760, 19)
대청호_2019.xlsx
(8760, 19)
(8760, 19)
옥천천_2016.xlsx
(8783, 10)
(8784, 10)
Make_columns
옥천천_2017.xlsx


/home/cjinw/work/tf-2/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # Remove the CWD from sys.path while we load stuff.


(8760, 10)
(8760, 10)
Make_columns
옥천천_2018.xlsx
(8754, 10)
(8760, 10)
Make_columns
옥천천_2019.xlsx
(8760, 10)
(8760, 10)
Make_columns
장계_2016.xlsx
(8764, 17)
(8784, 17)
장계_2017.xlsx
(8760, 17)
(8760, 17)
장계_2018.xlsx
(8759, 17)
(8760, 17)
장계_2019.xlsx
(8760, 17)
(8760, 17)


In [14]:
train_mean, train_std

(수온           0.900
 수소이온농도       6.300
 전기전도도       92.000
 용존산소         0.400
 총유기탄소        1.100
 총질소          0.174
 총인           0.000
 클로로필-a       0.000
 Day sin     -1.000
 Day cos     -1.000
 Year sin    -1.000
 Year cos    -1.000
 dtype: float64,
 수온           37.000
 수소이온농도       10.500
 전기전도도       558.000
 용존산소         29.300
 총유기탄소        12.900
 총질소           7.643
 총인            0.370
 클로로필-a      281.900
 Day sin       1.000
 Day cos       1.000
 Year sin      1.000
 Year cos      1.000
 dtype: float64)

In [15]:
df[0]

,수온,수소이온농도,전기전도도,용존산소,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,0.193906,0.142857,0.182403,0.162630,0.042373,NaN,NaN,0.012770,0.500000,1.000000,0.498667,0.999998
1,0.191136,0.142857,0.182403,0.159170,0.042373,NaN,NaN,0.012416,0.629410,0.982963,0.499025,0.999999
2,0.191136,0.142857,0.182403,0.155709,0.042373,NaN,NaN,0.012770,0.750000,0.933013,0.499384,1.000000
3,0.191136,0.142857,0.182403,0.155709,0.042373,NaN,NaN,0.012770,0.853553,0.853553,0.499742,1.000000
4,0.191136,0.142857,0.182403,0.155709,0.042373,NaN,NaN,0.012416,0.933013,0.750000,0.500100,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
35059,0.227147,0.190476,0.135193,0.228374,0.110169,0.128799,0.016216,0.013480,0.017037,0.629410,0.497133,0.999992
35060,0.227147,0.190476,0.133047,0.231834,0.110169,0.132414,0.021622,0.013835,0.066987,0.750000,0.497491,0.999994
35061,0.224377,0.190476,0.133047,0.231834,0.110169,0.134556,0.016216,0.013835,0.146447,0.853553,0.497850,0.999995
35062,0.221607,0.190476,0.133047,0.235294,0.110169,0.130673,0.021622,0.013835,0.250000,0.933013,0.498208,0.999997


In [16]:
# just for Create WidnowGenerator 
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [17]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
#                out_features = None,
               out_features = None, out_num_features = None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [18]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [19]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [20]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result
@property
def example2(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example2', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.val))
    # And cache it for next time
    self._example2 = result
  return result
@property
def example3(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example3', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example3 = result
  return result


WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example
WindowGenerator.example2 = example2
WindowGenerator.example3 = example3

In [21]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [22]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [23]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [24]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
#         print(group)
        n = np.max(group)
#         print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [25]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [26]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 miss_data_load_dir='save',
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)

        # Define mask matrix
        if miss_pattern is None:
            print("pattern none")
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            print("load save")
            self.miss = MissData(load_dir=miss_data_load_dir)
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

# dgen = GainDataGenerator(df)

In [27]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [28]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

# 측정망별 반복 구간 

In [29]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)
# 1: 자동, 2: 수질, 

[[    0     6     1     0]
 [    1    14     2     6]
 [    3    22     3    20]
 ...
 [ 6403    27     8 20235]
 [ 6411     7     7 20262]
 [ 6418    30    10 20269]]
miss_data file saved


In [30]:
dgen = GainDataGenerator(df)

pattern none


In [31]:
train_df = df_all
val_df = df_all
test_df = df_all
df_all

,수온,수소이온농도,전기전도도,용존산소,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,7.9,6.9,177.0,5.1,1.6,NaN,NaN,3.6,-1.407890e-12,1.000000,-0.002666,0.999996
1,7.8,6.9,177.0,5.0,1.6,NaN,NaN,3.5,2.588190e-01,0.965926,-0.001950,0.999998
2,7.8,6.9,177.0,4.9,1.6,NaN,NaN,3.6,5.000000e-01,0.866025,-0.001233,0.999999
3,7.8,6.9,177.0,4.9,1.6,NaN,NaN,3.6,7.071068e-01,0.707107,-0.000516,1.000000
4,7.8,6.9,177.0,4.9,1.6,NaN,NaN,3.5,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
35059,8.0,7.2,160.0,10.4,2.9,NaN,NaN,8.5,-9.659258e-01,0.258819,-0.005734,0.999984
35060,8.0,7.2,160.0,10.4,2.9,NaN,NaN,8.5,-8.660254e-01,0.500000,-0.005017,0.999987
35061,7.9,7.2,160.0,10.4,2.9,NaN,NaN,8.4,-7.071068e-01,0.707107,-0.004301,0.999991
35062,7.9,7.2,161.0,10.4,2.9,NaN,NaN,8.4,-5.000000e-01,0.866025,-0.003584,0.999994


In [32]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

## Graph 

In [33]:
wide_window.example[0].shape

load save


TensorShape([128, 120, 12])

In [34]:
wide_window.plot(plot_col=target_col)
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  12301


In [35]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [36]:
val_performance = {}
performance = {}

In [37]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [38]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [39]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

# gain.save(save_dir='save')

load save
load save
Epoch 1/2000
1/1 [==============================] - 1s 1s/step - gen_loss: 38.0033 - disc_loss: 0.7186 - rmse: 0.4928 - val_loss: 0.6778
Epoch 2/2000
1/1 [==============================] - 0s 146ms/step - gen_loss: 48.7911 - disc_loss: 0.5289 - rmse: 0.6786 - val_loss: 0.3214
Epoch 3/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 16.4345 - disc_loss: 0.4798 - rmse: 0.3162 - val_loss: 0.2688
Epoch 4/2000
1/1 [==============================] - 0s 144ms/step - gen_loss: 12.9594 - disc_loss: 0.4065 - rmse: 0.2652 - val_loss: 0.2310
Epoch 5/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 10.2015 - disc_loss: 0.3922 - rmse: 0.2321 - val_loss: 0.2023
Epoch 6/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 8.3510 - disc_loss: 0.3894 - rmse: 0.2005 - val_loss: 0.1786
Epoch 7/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 7.2364 - disc_loss: 0.3801 - rmse: 0.1820 - val_loss: 0.1581
Epoch

1/1 [==============================] - 0s 136ms/step - gen_loss: 0.3861 - disc_loss: 0.2391 - rmse: 0.0530 - val_loss: 0.0490
Epoch 118/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 0.3454 - disc_loss: 0.2313 - rmse: 0.0478 - val_loss: 0.0475
Epoch 119/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 0.3961 - disc_loss: 0.2419 - rmse: 0.0526 - val_loss: 0.0475
Epoch 120/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 0.3409 - disc_loss: 0.2303 - rmse: 0.0498 - val_loss: 0.0497
Epoch 121/2000
1/1 [==============================] - 0s 106ms/step - gen_loss: 0.3370 - disc_loss: 0.2415 - rmse: 0.0477 - val_loss: 0.0446
Epoch 122/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 0.3646 - disc_loss: 0.2293 - rmse: 0.0504 - val_loss: 0.0459
Epoch 123/2000
1/1 [==============================] - 0s 151ms/step - gen_loss: 0.3500 - disc_loss: 0.2292 - rmse: 0.0468 - val_loss: 0.0448
Epoch 124/2000
1/1 [========

1/1 [==============================] - 0s 163ms/step - gen_loss: 0.4021 - disc_loss: 0.1895 - rmse: 0.0432 - val_loss: 0.0386
Epoch 234/2000
1/1 [==============================] - 0s 148ms/step - gen_loss: 0.4023 - disc_loss: 0.1880 - rmse: 0.0438 - val_loss: 0.0397
Epoch 235/2000
1/1 [==============================] - 0s 150ms/step - gen_loss: 0.3720 - disc_loss: 0.1923 - rmse: 0.0409 - val_loss: 0.0445
Epoch 236/2000
1/1 [==============================] - 0s 122ms/step - gen_loss: 0.3802 - disc_loss: 0.1884 - rmse: 0.0392 - val_loss: 0.0475
Epoch 237/2000
1/1 [==============================] - 0s 104ms/step - gen_loss: 0.3877 - disc_loss: 0.1950 - rmse: 0.0371 - val_loss: 0.0463
Epoch 238/2000
1/1 [==============================] - 0s 146ms/step - gen_loss: 0.3525 - disc_loss: 0.1895 - rmse: 0.0385 - val_loss: 0.0481
Epoch 239/2000
1/1 [==============================] - 0s 161ms/step - gen_loss: 0.3777 - disc_loss: 0.1861 - rmse: 0.0376 - val_loss: 0.0415
Epoch 240/2000
1/1 [========

1/1 [==============================] - 0s 112ms/step - gen_loss: 0.4206 - disc_loss: 0.1688 - rmse: 0.0374 - val_loss: 0.0481
Epoch 350/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 0.4254 - disc_loss: 0.1738 - rmse: 0.0385 - val_loss: 0.0370
Epoch 351/2000
1/1 [==============================] - 0s 108ms/step - gen_loss: 0.4271 - disc_loss: 0.1725 - rmse: 0.0356 - val_loss: 0.0380
Epoch 352/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 0.4197 - disc_loss: 0.1707 - rmse: 0.0363 - val_loss: 0.0383
Epoch 353/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 0.4204 - disc_loss: 0.1699 - rmse: 0.0348 - val_loss: 0.0376
Epoch 354/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 0.4375 - disc_loss: 0.1726 - rmse: 0.0361 - val_loss: 0.0413
Epoch 355/2000
1/1 [==============================] - 0s 141ms/step - gen_loss: 0.4332 - disc_loss: 0.1718 - rmse: 0.0343 - val_loss: 0.0348
Epoch 356/2000
1/1 [========

1/1 [==============================] - 0s 138ms/step - gen_loss: 0.3729 - disc_loss: 0.1665 - rmse: 0.0404 - val_loss: 0.0366
Epoch 466/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 0.3886 - disc_loss: 0.1602 - rmse: 0.0355 - val_loss: 0.0329
Epoch 467/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 0.4162 - disc_loss: 0.1611 - rmse: 0.0365 - val_loss: 0.0333
Epoch 468/2000
1/1 [==============================] - 0s 140ms/step - gen_loss: 0.4067 - disc_loss: 0.1630 - rmse: 0.0338 - val_loss: 0.0305
Epoch 469/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 0.3519 - disc_loss: 0.1573 - rmse: 0.0298 - val_loss: 0.0364
Epoch 470/2000
1/1 [==============================] - 0s 120ms/step - gen_loss: 0.3602 - disc_loss: 0.1629 - rmse: 0.0319 - val_loss: 0.0365
Epoch 471/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 0.4233 - disc_loss: 0.1572 - rmse: 0.0325 - val_loss: 0.0327
Epoch 472/2000
1/1 [========

...

1/1 [==============================] - 0s 120ms/step - gen_loss: 0.5188 - disc_loss: 0.1156 - rmse: 0.0223 - val_loss: 0.0239
Epoch 1503/2000
1/1 [==============================] - 0s 145ms/step - gen_loss: 0.4930 - disc_loss: 0.1134 - rmse: 0.0240 - val_loss: 0.0236
Epoch 1504/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 0.5280 - disc_loss: 0.1172 - rmse: 0.0240 - val_loss: 0.0232
Epoch 1505/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 0.5104 - disc_loss: 0.1130 - rmse: 0.0227 - val_loss: 0.0255
Epoch 1506/2000
1/1 [==============================] - 0s 110ms/step - gen_loss: 0.4790 - disc_loss: 0.1147 - rmse: 0.0221 - val_loss: 0.0219
Epoch 1507/2000
1/1 [==============================] - 0s 145ms/step - gen_loss: 0.5340 - disc_loss: 0.1151 - rmse: 0.0230 - val_loss: 0.0248
Epoch 1508/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 0.4948 - disc_loss: 0.1179 - rmse: 0.0239 - val_loss: 0.0244
Epoch 1509/2000
1/1 [=

1/1 [==============================] - 0s 150ms/step - gen_loss: 0.5819 - disc_loss: 0.1169 - rmse: 0.0261 - val_loss: 0.0230
Epoch 1617/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 0.5805 - disc_loss: 0.1152 - rmse: 0.0229 - val_loss: 0.0246
Epoch 1618/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 0.5416 - disc_loss: 0.1151 - rmse: 0.0231 - val_loss: 0.0247
Epoch 1619/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 0.6216 - disc_loss: 0.1181 - rmse: 0.0282 - val_loss: 0.0299
Epoch 1620/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 0.5919 - disc_loss: 0.1134 - rmse: 0.0224 - val_loss: 0.0295
Epoch 1621/2000
1/1 [==============================] - 0s 149ms/step - gen_loss: 0.5935 - disc_loss: 0.1124 - rmse: 0.0268 - val_loss: 0.0243
Epoch 1622/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 0.5578 - disc_loss: 0.1151 - rmse: 0.0225 - val_loss: 0.0256
Epoch 1623/2000
1/1 [=

1/1 [==============================] - 0s 480ms/step - gen_loss: 0.5203 - disc_loss: 0.1083 - rmse: 0.0220 - val_loss: 0.0232
Epoch 1731/2000
1/1 [==============================] - 0s 125ms/step - gen_loss: 0.5459 - disc_loss: 0.1122 - rmse: 0.0258 - val_loss: 0.0222
Epoch 1732/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 0.5318 - disc_loss: 0.1127 - rmse: 0.0222 - val_loss: 0.0237
Epoch 1733/2000
1/1 [==============================] - 0s 151ms/step - gen_loss: 0.5078 - disc_loss: 0.1114 - rmse: 0.0241 - val_loss: 0.0243
Epoch 1734/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 0.5802 - disc_loss: 0.1132 - rmse: 0.0215 - val_loss: 0.0239
Epoch 1735/2000
1/1 [==============================] - 0s 151ms/step - gen_loss: 0.5371 - disc_loss: 0.1111 - rmse: 0.0242 - val_loss: 0.0269
Epoch 1736/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 0.5238 - disc_loss: 0.1119 - rmse: 0.0236 - val_loss: 0.0236
Epoch 1737/2000
1/1 [=

Epoch 1788/2000
1/1 [==============================] - 0s 141ms/step - gen_loss: 0.5281 - disc_loss: 0.1131 - rmse: 0.0223 - val_loss: 0.0236
Epoch 1789/2000
1/1 [==============================] - 0s 137ms/step - gen_loss: 0.5969 - disc_loss: 0.1129 - rmse: 0.0234 - val_loss: 0.0232
Epoch 1790/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 0.5293 - disc_loss: 0.1103 - rmse: 0.0233 - val_loss: 0.0204
Epoch 1791/2000
1/1 [==============================] - 0s 137ms/step - gen_loss: 0.5264 - disc_loss: 0.1083 - rmse: 0.0229 - val_loss: 0.0220
Epoch 1792/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 0.5465 - disc_loss: 0.1111 - rmse: 0.0240 - val_loss: 0.0231
Epoch 1793/2000
1/1 [==============================] - 0s 128ms/step - gen_loss: 0.5448 - disc_loss: 0.1090 - rmse: 0.0230 - val_loss: 0.0238
Epoch 1794/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 0.5205 - disc_loss: 0.1065 - rmse: 0.0234 - val_loss: 0.0226
Epoch 

1/1 [==============================] - 0s 136ms/step - gen_loss: 0.5418 - disc_loss: 0.1149 - rmse: 0.0228 - val_loss: 0.0209
Epoch 1903/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 0.4986 - disc_loss: 0.1081 - rmse: 0.0218 - val_loss: 0.0223
Epoch 1904/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 0.4899 - disc_loss: 0.1107 - rmse: 0.0236 - val_loss: 0.0210
Epoch 1905/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 0.5871 - disc_loss: 0.1101 - rmse: 0.0256 - val_loss: 0.0226
Epoch 1906/2000
1/1 [==============================] - 0s 122ms/step - gen_loss: 0.5034 - disc_loss: 0.1101 - rmse: 0.0220 - val_loss: 0.0226
Epoch 1907/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 0.5372 - disc_loss: 0.1098 - rmse: 0.0250 - val_loss: 0.0227
Epoch 1908/2000
1/1 [==============================] - 0s 154ms/step - gen_loss: 0.5774 - disc_loss: 0.1140 - rmse: 0.0216 - val_loss: 0.0259
Epoch 1909/2000
1/1 [=

**학습 loss history 출력**

In [40]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**성능 측정**

In [41]:
gain.evaluate(wide_window.test.repeat(), steps=100)

load save
100/100 [==============================] - 3s 32ms/step - loss: 0.0245


0.02452436275780201

**model save**

In [42]:
gain.save(save_dir='save')

**샘플 prediction 출력**

In [43]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
norm_df = pd.concat(df,axis=0)

In [45]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

140256
(120, 12)
120
x.shape = (140160, 12)
x_reshape.shape = (1168, 120, 12)
169322
y_true.shape= (140160, 12)
169322


(1, 120, 12)

In [46]:
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

37/37 [==============================] - 0s 2ms/step - loss: 0.0000e+00


0.0

In [47]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [48]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(140160, 12) (120, 12)
(140256, 12)


In [49]:
y_pred[~np.isnan(data)] = np.nan

In [50]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Imputed Data**

In [51]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

140256
unit_shape= (120, 12)
120
n= 140160
(35040, 8)
(35040, 8)


# 예측

In [52]:
df[0].head

<bound method NDFrame.head of              수온    수소이온농도     전기전도도      용존산소     총유기탄소       총질소        총인  \
0      0.193906  0.142857  0.182403  0.162630  0.042373       NaN       NaN   
1      0.191136  0.142857  0.182403  0.159170  0.042373       NaN       NaN   
2      0.191136  0.142857  0.182403  0.155709  0.042373       NaN       NaN   
3      0.191136  0.142857  0.182403  0.155709  0.042373       NaN       NaN   
4      0.191136  0.142857  0.182403  0.155709  0.042373       NaN       NaN   
...         ...       ...       ...       ...       ...       ...       ...   
35059  0.227147  0.190476  0.135193  0.228374  0.110169  0.128799  0.016216   
35060  0.227147  0.190476  0.133047  0.231834  0.110169  0.132414  0.021622   
35061  0.224377  0.190476  0.133047  0.231834  0.110169  0.134556  0.016216   
35062  0.221607  0.190476  0.133047  0.235294  0.110169  0.130673  0.021622   
35063  0.224377  0.190476  0.133047  0.231834  0.110169  0.129736  0.021622   

         클로로필-a   Day

In [53]:
oris[1].shape

(35040, 8)

In [54]:
    ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
# print(oris[0].shape, gans[0].shape)
# print(oris[1].shape, gans[1].shape)
# print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
# out_features = [target_col_idx]
# out_num_features = len(out_features)
# out_num_features

In [55]:
train_df 

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.193906,0.142857,0.182403,0.162630,0.042373,0.173954,0.023721,0.012770,0.500000,1.000000,...,0.010811,0.034260,0.091413,0.309524,0.214592,0.439446,0.050847,0.330566,0.056087,0.007449
1,0.191136,0.142857,0.182403,0.159170,0.042373,0.169891,0.017134,0.012416,0.629410,0.982963,...,0.010811,0.037273,0.091413,0.309524,0.214592,0.439446,0.042373,0.318383,0.050422,0.007804
2,0.191136,0.142857,0.182403,0.155709,0.042373,0.141038,0.018956,0.012770,0.750000,0.933013,...,0.013514,0.000751,0.091413,0.333333,0.214592,0.442907,0.050847,0.316374,0.029072,0.007095
3,0.191136,0.142857,0.182403,0.155709,0.042373,0.136993,0.036406,0.012770,0.853553,0.853553,...,0.013514,0.056208,0.094183,0.333333,0.214592,0.439446,0.042373,0.320257,0.024262,0.006740
4,0.191136,0.142857,0.182403,0.155709,0.042373,0.170429,0.030826,0.012416,0.933013,0.750000,...,0.010811,0.009035,0.091413,0.309524,0.214592,0.439446,0.042373,0.323872,0.041494,0.007449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24523,0.504155,0.166667,0.085837,0.155709,0.135593,0.166421,0.086486,0.023413,0.017037,0.629410,...,0.071154,0.036553,0.523546,0.285714,0.126609,0.294118,0.178373,0.179274,0.056757,0.080525
24524,0.501385,0.166667,0.085837,0.159170,0.135593,0.157719,0.089189,0.023767,0.066987,0.750000,...,0.061121,0.024263,0.523546,0.285714,0.126609,0.287197,0.183698,0.178337,0.056757,0.078042
24525,0.501385,0.166667,0.085837,0.152249,0.135593,0.162137,0.089189,0.024122,0.146447,0.853553,...,0.074976,0.025770,0.520776,0.261905,0.126609,0.280277,0.181091,0.181417,0.059459,0.074495
24526,0.498615,0.166667,0.085837,0.148789,0.135593,0.161334,0.089189,0.023767,0.250000,0.933013,...,0.061021,0.027890,0.520776,0.238095,0.126609,0.273356,0.182634,0.199357,0.056757,0.073076


In [56]:
df[0].columns

Index(['수온', '수소이온농도', '전기전도도', '용존산소', '총유기탄소', '총질소', '총인', '클로로필-a',
       'Day sin', 'Day cos', 'Year sin', 'Year cos'],
      dtype='object')

In [57]:
MAX_EPOCHS = 150
target_col = '용존산소'
num_features = df[0].shape[1]
label_columns_indices = {name: i for i, name in enumerate(df[0])}
target_col_idx = label_columns_indices[target_col]
# target_col_idx
# out_features = [target_col_idx]
out_features = [3,4,5,6,7]
out_num_features = len(out_features)
out_num_features

5

In [58]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
#         print("out_features")
#         print(out_features)
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [59]:
OUT_STEPS = 24*3

In [60]:
# OUT_STEPS = 24*3
# wdg = WaterDataGenerator(train_df,
#                          batch_size=128,
#                          input_width = 24*7,
#                          label_width = OUT_STEPS,
#                          shift = OUT_STEPS,
#                          out_num_features = out_num_features,
# #                          out_features = out_features
#                         )

In [61]:
# train_df

In [62]:
# it = iter(wdg)

In [63]:
# x,y = next(it)
# x.shape, y.shape

In [64]:
def make_dataset_water(self, data):
#   print("make_dataset_water")
#   print(out_features)
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [65]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

**1일단위로 변환**

In [66]:
def hour_to_day_mean(array):
    
    time = 24
    return tf.reduce_mean(tf.reshape(array, [array.shape[0]//time, time]), 1)

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
    
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = (input_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = (label_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = (predict_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#       predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

**예측률 계산**

In [67]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [68]:
def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0, windows=None, min_max_normailze=False):
       
    if example is not None :
        inputs, labels = windows
    else :
        inputs, labels = self.example
        
        
    plot_out_col_index = self.column_indices[plot_out_col]
    
    if self.label_columns:
        label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
        label_out_col_index = plot_out_col_index
    
    if model is None:
        return
    
    print("column : " + df_all.columns[plot_col])
    
    mae = 0
    mse = 0
    rmse = 0
    mape = 0
    pred_arr = []
    label_arr = []
    
    predictions = model(inputs)
    
    if min_max_normailze :
        predictions = predictions * train_std[plot_col] + train_mean[plot_col]
        labels = labels * train_std[plot_col] + train_mean[plot_col]
        
        predictions = predictions.numpy()
        labels = labels.numpy()
        
        predictions = (predictions - predictions.min()) / (predictions.max() - predictions.min())
        labels = (labels - labels.min()) / (labels.max() - labels.min())

    for n in range(len(inputs)):
        pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
        label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
        
        pred_arr.append(pred_temp[2])
        label_arr.append(label_temp[2])
        
        
    mae = mean_absolute_error(label_arr, pred_arr)
    mse = mean_squared_error(label_arr, pred_arr)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((np.array(label_arr) - np.array(pred_arr))/np.array(label_arr)) )*100
    
    print("mae:")  
    print(mae)
    
    print("mse:")  
    print(mse)
    
    print("rmse")  
    print(rmse)
    
    print("mape")  
    print(mape)
    
    

WindowGenerator.compa = compa

In [69]:
train_df.shape

(24528, 36)

In [70]:
#multi_window.compa(elman_model, plot_col=out_features[0])

In [71]:
# target_col = '용존산소'
# num_features = df[0].shape[1]
# label_columns_indices = {name: i for i, name in enumerate(df[0])}
# target_col_idx = label_columns_indices[target_col]
# # target_col_idx
# out_features = [target_col_idx]
# # out_features = [3,5,6,7,8]
# out_num_features = len(out_features)
# out_num_features

In [72]:
def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [73]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

In [74]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
#                                out_features = out_features,
#                                out_num_features = out_num_features,
                               )

In [75]:
multi_val_performance = {}
multi_performance = {}

**Elman_RNN**

In [76]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [77]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [78]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

Epoch 1/150
1/1 [==============================] - 1s 839ms/step - loss: 0.0234 - mean_absolute_error: 0.1121 - val_loss: 0.0139 - val_mean_absolute_error: 0.0874
Epoch 2/150
1/1 [==============================] - 0s 324ms/step - loss: 0.0158 - mean_absolute_error: 0.0867 - val_loss: 0.0080 - val_mean_absolute_error: 0.0649
Epoch 3/150
1/1 [==============================] - 0s 285ms/step - loss: 0.0097 - mean_absolute_error: 0.0667 - val_loss: 0.0030 - val_mean_absolute_error: 0.0382
Epoch 4/150
1/1 [==============================] - 0s 309ms/step - loss: 0.0052 - mean_absolute_error: 0.0511 - val_loss: 0.0026 - val_mean_absolute_error: 0.0413
Epoch 5/150
1/1 [==============================] - 0s 299ms/step - loss: 0.0049 - mean_absolute_error: 0.0551 - val_loss: 0.0033 - val_mean_absolute_error: 0.0433
Epoch 6/150
1/1 [==============================] - 0s 391ms/step - loss: 0.0051 - mean_absolute_error: 0.0537 - val_loss: 0.0026 - val_mean_absolute_error: 0.0340
Epoch 7/150
1/1 [=====

Epoch 50/150
1/1 [==============================] - 0s 271ms/step - loss: 4.9525e-04 - mean_absolute_error: 0.0156 - val_loss: 8.4013e-04 - val_mean_absolute_error: 0.0215
Epoch 51/150
1/1 [==============================] - 0s 345ms/step - loss: 6.1197e-04 - mean_absolute_error: 0.0166 - val_loss: 8.0621e-04 - val_mean_absolute_error: 0.0218
Epoch 52/150
1/1 [==============================] - 0s 303ms/step - loss: 5.5792e-04 - mean_absolute_error: 0.0170 - val_loss: 8.2958e-04 - val_mean_absolute_error: 0.0221
Epoch 53/150
1/1 [==============================] - 0s 258ms/step - loss: 5.6337e-04 - mean_absolute_error: 0.0169 - val_loss: 7.6833e-04 - val_mean_absolute_error: 0.0209
Epoch 54/150
1/1 [==============================] - 0s 300ms/step - loss: 6.7365e-04 - mean_absolute_error: 0.0180 - val_loss: 7.4884e-04 - val_mean_absolute_error: 0.0206
Epoch 55/150
1/1 [==============================] - 0s 292ms/step - loss: 5.7937e-04 - mean_absolute_error: 0.0167 - val_loss: 6.6039e-04 - 

Epoch 98/150
1/1 [==============================] - 0s 289ms/step - loss: 3.8480e-04 - mean_absolute_error: 0.0143 - val_loss: 4.9424e-04 - val_mean_absolute_error: 0.0170
Epoch 99/150
1/1 [==============================] - 0s 311ms/step - loss: 3.7337e-04 - mean_absolute_error: 0.0139 - val_loss: 4.8347e-04 - val_mean_absolute_error: 0.0169
Epoch 100/150
1/1 [==============================] - 0s 334ms/step - loss: 3.2884e-04 - mean_absolute_error: 0.0136 - val_loss: 4.5077e-04 - val_mean_absolute_error: 0.0161
Epoch 101/150
1/1 [==============================] - 0s 284ms/step - loss: 3.7311e-04 - mean_absolute_error: 0.0134 - val_loss: 6.1984e-04 - val_mean_absolute_error: 0.0180
Epoch 102/150
1/1 [==============================] - 0s 300ms/step - loss: 4.0475e-04 - mean_absolute_error: 0.0141 - val_loss: 4.7511e-04 - val_mean_absolute_error: 0.0166
Epoch 103/150
1/1 [==============================] - 0s 290ms/step - loss: 3.6587e-04 - mean_absolute_error: 0.0137 - val_loss: 4.9803e-0

In [79]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [80]:
multi_window.plot2(elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**GRU**

In [81]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [82]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/150
1/1 [==============================] - 1s 937ms/step - loss: 0.0236 - mean_absolute_error: 0.1129 - val_loss: 0.0184 - val_mean_absolute_error: 0.0999
Epoch 2/150
1/1 [==============================] - 0s 292ms/step - loss: 0.0206 - mean_absolute_error: 0.1013 - val_loss: 0.0157 - val_mean_absolute_error: 0.0911
Epoch 3/150
1/1 [==============================] - 0s 318ms/step - loss: 0.0192 - mean_absolute_error: 0.0927 - val_loss: 0.0133 - val_mean_absolute_error: 0.0844
Epoch 4/150
1/1 [==============================] - 0s 353ms/step - loss: 0.0153 - mean_absolute_error: 0.0825 - val_loss: 0.0100 - val_mean_absolute_error: 0.0725
Epoch 5/150
1/1 [==============================] - 0s 344ms/step - loss: 0.0116 - mean_absolute_error: 0.0716 - val_loss: 0.0078 - val_mean_absolute_error: 0.0612
Epoch 6/150
1/1 [==============================] - 0s 339ms/step - loss: 0.0085 - mean_absolute_error: 0.0620 - val_loss: 0.0047 - val_mean_absolute_error: 0.0467
Epoch 7/150
1/1 [=====

Epoch 98/150
1/1 [==============================] - 0s 338ms/step - loss: 5.9878e-04 - mean_absolute_error: 0.0169 - val_loss: 6.2711e-04 - val_mean_absolute_error: 0.0190
Epoch 99/150
1/1 [==============================] - 0s 310ms/step - loss: 6.1235e-04 - mean_absolute_error: 0.0167 - val_loss: 6.7472e-04 - val_mean_absolute_error: 0.0195
Epoch 100/150
1/1 [==============================] - 0s 336ms/step - loss: 6.1327e-04 - mean_absolute_error: 0.0172 - val_loss: 6.9103e-04 - val_mean_absolute_error: 0.0198
Epoch 101/150
1/1 [==============================] - 0s 355ms/step - loss: 5.9528e-04 - mean_absolute_error: 0.0168 - val_loss: 7.0688e-04 - val_mean_absolute_error: 0.0203
Epoch 102/150
1/1 [==============================] - 0s 334ms/step - loss: 4.8188e-04 - mean_absolute_error: 0.0158 - val_loss: 6.8540e-04 - val_mean_absolute_error: 0.0201
Epoch 103/150
1/1 [==============================] - 0s 342ms/step - loss: 5.7680e-04 - mean_absolute_error: 0.0166 - val_loss: 6.6915e-0

**LSTM**

In [83]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [84]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/150
1/1 [==============================] - 3s 3s/step - loss: 0.0232 - mean_absolute_error: 0.1106 - val_loss: 0.0188 - val_mean_absolute_error: 0.1008
Epoch 2/150
1/1 [==============================] - 0s 279ms/step - loss: 0.0214 - mean_absolute_error: 0.1035 - val_loss: 0.0161 - val_mean_absolute_error: 0.0925
Epoch 3/150
1/1 [==============================] - 0s 262ms/step - loss: 0.0189 - mean_absolute_error: 0.0927 - val_loss: 0.0140 - val_mean_absolute_error: 0.0859
Epoch 4/150
1/1 [==============================] - 0s 245ms/step - loss: 0.0164 - mean_absolute_error: 0.0859 - val_loss: 0.0104 - val_mean_absolute_error: 0.0743
Epoch 5/150
1/1 [==============================] - 0s 275ms/step - loss: 0.0116 - mean_absolute_error: 0.0730 - val_loss: 0.0063 - val_mean_absolute_error: 0.0562
Epoch 6/150
1/1 [==============================] - 0s 257ms/step - loss: 0.0089 - mean_absolute_error: 0.0621 - val_loss: 0.0039 - val_mean_absolute_error: 0.0445
Epoch 7/150
1/1 [========

Epoch 98/150
1/1 [==============================] - 0s 274ms/step - loss: 4.6023e-04 - mean_absolute_error: 0.0150 - val_loss: 5.6231e-04 - val_mean_absolute_error: 0.0182
Epoch 99/150
1/1 [==============================] - 0s 266ms/step - loss: 4.2789e-04 - mean_absolute_error: 0.0149 - val_loss: 5.6756e-04 - val_mean_absolute_error: 0.0186
Epoch 100/150
1/1 [==============================] - 0s 273ms/step - loss: 4.3710e-04 - mean_absolute_error: 0.0149 - val_loss: 5.0997e-04 - val_mean_absolute_error: 0.0177
Epoch 101/150
1/1 [==============================] - 0s 281ms/step - loss: 4.2764e-04 - mean_absolute_error: 0.0141 - val_loss: 5.9419e-04 - val_mean_absolute_error: 0.0189
Epoch 102/150
1/1 [==============================] - 0s 272ms/step - loss: 4.4936e-04 - mean_absolute_error: 0.0143 - val_loss: 6.1689e-04 - val_mean_absolute_error: 0.0188
Epoch 103/150
1/1 [==============================] - 0s 248ms/step - loss: 4.0315e-04 - mean_absolute_error: 0.0140 - val_loss: 5.8940e-0

**CNN model**

In [85]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [86]:
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

Epoch 1/150
1/1 [==============================] - 1s 775ms/step - loss: 0.0227 - mean_absolute_error: 0.1115 - val_loss: 0.0150 - val_mean_absolute_error: 0.0886
Epoch 2/150
1/1 [==============================] - 0s 232ms/step - loss: 0.0178 - mean_absolute_error: 0.0883 - val_loss: 0.0064 - val_mean_absolute_error: 0.0598
Epoch 3/150
1/1 [==============================] - 0s 262ms/step - loss: 0.0073 - mean_absolute_error: 0.0622 - val_loss: 0.0059 - val_mean_absolute_error: 0.0647
Epoch 4/150
1/1 [==============================] - 0s 249ms/step - loss: 0.0086 - mean_absolute_error: 0.0756 - val_loss: 0.0040 - val_mean_absolute_error: 0.0491
Epoch 5/150
1/1 [==============================] - 0s 254ms/step - loss: 0.0059 - mean_absolute_error: 0.0591 - val_loss: 0.0018 - val_mean_absolute_error: 0.0310
Epoch 6/150
1/1 [==============================] - 0s 252ms/step - loss: 0.0042 - mean_absolute_error: 0.0478 - val_loss: 0.0024 - val_mean_absolute_error: 0.0391
Epoch 7/150
1/1 [=====

Epoch 50/150
1/1 [==============================] - 0s 246ms/step - loss: 4.6892e-04 - mean_absolute_error: 0.0158 - val_loss: 6.0691e-04 - val_mean_absolute_error: 0.0188
Epoch 51/150
1/1 [==============================] - 0s 257ms/step - loss: 4.3277e-04 - mean_absolute_error: 0.0151 - val_loss: 6.1289e-04 - val_mean_absolute_error: 0.0189
Epoch 52/150
1/1 [==============================] - 0s 275ms/step - loss: 4.6434e-04 - mean_absolute_error: 0.0156 - val_loss: 6.0736e-04 - val_mean_absolute_error: 0.0187
Epoch 53/150
1/1 [==============================] - 0s 248ms/step - loss: 5.2846e-04 - mean_absolute_error: 0.0153 - val_loss: 6.2406e-04 - val_mean_absolute_error: 0.0189
Epoch 54/150
1/1 [==============================] - 0s 247ms/step - loss: 5.0679e-04 - mean_absolute_error: 0.0155 - val_loss: 5.9707e-04 - val_mean_absolute_error: 0.0187
Epoch 55/150
1/1 [==============================] - 0s 272ms/step - loss: 4.6845e-04 - mean_absolute_error: 0.0152 - val_loss: 5.6874e-04 - 

Epoch 98/150
1/1 [==============================] - 0s 253ms/step - loss: 3.0777e-04 - mean_absolute_error: 0.0126 - val_loss: 3.9562e-04 - val_mean_absolute_error: 0.0150
Epoch 99/150
1/1 [==============================] - 0s 250ms/step - loss: 2.8753e-04 - mean_absolute_error: 0.0122 - val_loss: 4.6233e-04 - val_mean_absolute_error: 0.0162
Epoch 100/150
1/1 [==============================] - 0s 248ms/step - loss: 3.1444e-04 - mean_absolute_error: 0.0123 - val_loss: 4.1822e-04 - val_mean_absolute_error: 0.0156
Epoch 101/150
1/1 [==============================] - 0s 251ms/step - loss: 3.0499e-04 - mean_absolute_error: 0.0124 - val_loss: 4.3261e-04 - val_mean_absolute_error: 0.0156
Epoch 102/150
1/1 [==============================] - 0s 284ms/step - loss: 3.0836e-04 - mean_absolute_error: 0.0125 - val_loss: 4.1233e-04 - val_mean_absolute_error: 0.0156
Epoch 103/150
1/1 [==============================] - 0s 268ms/step - loss: 2.5779e-04 - mean_absolute_error: 0.0117 - val_loss: 4.0459e-0

In [87]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

1/1 [==============================] - 0s 137ms/step - loss: 4.5238e-04 - mean_absolute_error: 0.0158
val performance = [0.00045238083112053573, 0.01581994630396366]
test performance =  [0.0002893652126658708, 0.012092825025320053]


**performance**

In [88]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [96]:
multi_window.compa(elman_model, plot_col=out_features[4], windows=multi_window.example2, min_max_normailze=False)

column : 클로로필-a
mae:
0.022216428
mse:
0.0007126845
rmse
0.026696151
mape
10.725874453783035


In [ ]:
multi_window.compa(gru_model, plot_col=out_features[1], windows=multi_window.example2, min_max_normailze=False)

column : 총유기탄소
mae:
0.01947055
mse:
0.00061043404
rmse
0.024706963
mape
9.932716190814972


In [91]:
multi_window.compa(multi_lstm_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=False)

column : 용존산소
mae:
0.021890443
mse:
0.00079140405
rmse
0.028131904
mape
10.075739026069641


In [92]:
multi_window.compa(multi_conv_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=False)

column : 용존산소
mae:
0.01893247
mse:
0.00056459475
rmse
0.023761204
mape
8.48381370306015


In [93]:
multi_window.plot24(gru_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …